Working With Text Data
===

- https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [24]:
import pandas as pd
import numpy as np
import joblib

In [2]:
#urls = 'https://github.com/dragon-library/work_space/raw/main/HS_Code/HS/hs_code.xlsx'
urls = 'data/hs_code.xlsx'
types = 'section'


def get_master(sheets, types='section'):
    data = pd.read_excel(urls, sheet_name=sheets)
    data[types] = data[types].map('{:02}'.format)
    data = data[[types, 'description']]
    data['description'] = data['description'].str.lower()
    #   data = data.rename(columns={'heading' : 'target', 'product_desc' : 'question_text'})

    return data


def manage_data(df):
    df.columns = ['target', 'data']

    return df

In [34]:
import joblib
files = "../data/train/df_master_section.pkl"
print("Load DataFrame")
twenty_train = joblib.load(files)
#df.columns =  ['Conference', 'Title'] 
twenty_train = twenty_train[['section', 'description']]
twenty_train.columns = ['target', 'data']
print(len(twenty_train))
twenty_train.head()

Load DataFrame
49134


,target,data
0,01,"horses; live, purebred breeding animals - pure..."
1,01,"horses; live, other than purebred breeding ani..."
2,01,asses; live - other
3,01,mules and hinnies; live- other
4,01,"cattle; live, purebred breeding animals - pure..."


In [35]:
from collections import Counter
Counter(twenty_train["target"])

Counter({'01': 1721,
         '02': 1487,
         '03': 327,
         '04': 2208,
         '05': 691,
         '06': 5045,
         '07': 2868,
         '08': 483,
         '09': 745,
         '10': 901,
         '11': 4048,
         '12': 394,
         '13': 992,
         '14': 590,
         '15': 10406,
         '16': 8452,
         '17': 5134,
         '18': 1467,
         '19': 89,
         '20': 1053,
         '21': 33})

In [5]:
types = "section"


sheets = "deci"
deci = get_master(sheets,types)



twenty_test = manage_data(deci)

print(len(twenty_test.data))
twenty_test.head()

NameError: name 'get_master' is not defined

In [36]:
twenty_train["target"].value_counts(normalize = True)

15    0.211788
16    0.172019
17    0.104490
06    0.102678
11    0.082387
07    0.058371
04    0.044938
01    0.035027
02    0.030264
18    0.029857
20    0.021431
13    0.020190
10    0.018338
09    0.015163
05    0.014064
14    0.012008
08    0.009830
12    0.008019
03    0.006655
19    0.001811
21    0.000672
Name: target, dtype: float64

In [32]:
twenty_train["target"] 

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
49131   NaN
49132   NaN
49133   NaN
49134   NaN
49135   NaN
Name: target, Length: 49134, dtype: float64

In [8]:
 twenty_test.target.unique()

array(['17', '06', '18', '16', '04', '20', '07', '10', '15', '05', '13',
       '11', '09', '03'], dtype=object)

In [11]:

category =  twenty_train.target.unique()

twenty_train.target_names =  twenty_train.target.unique()
twenty_test.target_name =  twenty_test.target.unique()

C:\Users\ASUS\.conda\envs\workspace\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  This is separate from the ipykernel package so we can avoid doing imports until


NameError: name 'twenty_test' is not defined

In [12]:
len(twenty_train.data)

49134

In [13]:
twenty_train.target[:10]

0    01
1    01
2    01
3    01
4    01
5    01
6    01
7    01
8    01
9    01
Name: target, dtype: object

In [14]:
for t in twenty_train.target[:10]:
    a = int(t)
    print(twenty_train.target[a])

01
01
01
01
01
01
01
01
01
01


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
>>> count_vect = CountVectorizer()
>>> X_train_counts = count_vect.fit_transform(twenty_train.data)
>>> X_train_counts.shape

(49134, 32497)

In [38]:
count_vect.vocabulary_.get(u'algorithm')

In [39]:
>>> from sklearn.feature_extraction.text import TfidfTransformer
>>> tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
>>> X_train_tf = tf_transformer.transform(X_train_counts)
>>> X_train_tf.shape

(49134, 32497)

In [40]:
>>> tfidf_transformer = TfidfTransformer()
>>> X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
>>> X_train_tfidf.shape

(49134, 32497)

In [41]:
>>> from sklearn.naive_bayes import MultinomialNB
>>> clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

In [44]:
>>> docs_new = ["Textile fabrics","Horse"]
>>> X_new_counts = count_vect.transform(docs_new)
>>> X_new_tfidf = tfidf_transformer.transform(X_new_counts)

>>> predicted = clf.predict(X_new_tfidf)

#>>> for doc, category in zip(docs_new, predicted):
#...     print('%r => %s' % (doc, twenty_train.target_names[int(category)]))

In [45]:
>>> from sklearn.pipeline import Pipeline
>>> text_clf = Pipeline([
...     ('vect', CountVectorizer()),
...     ('tfidf', TfidfTransformer()),
...     ('clf', MultinomialNB()),
... ])

In [46]:
text_clf.fit(twenty_train.data, twenty_train.target)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [47]:
>>> import numpy as np

>>> docs_test = twenty_test.data
>>> predicted = text_clf.predict(docs_test)
>>> np.mean(predicted == twenty_test.target)

NameError: name 'twenty_test' is not defined

In [22]:
>>> from sklearn.linear_model import SGDClassifier
>>> text_clf = Pipeline([
...     ('vect', CountVectorizer()),
...     ('tfidf', TfidfTransformer()),
...     ('clf', SGDClassifier(loss='hinge', penalty='l2',
...                           alpha=1e-3, random_state=42,
...                           max_iter=5, tol=None)),
... ])

>>> text_clf.fit(twenty_train.data, twenty_train.target)

>>> predicted = text_clf.predict(docs_test)
>>> np.mean(predicted == twenty_test.target)

0.5909869773817683

In [23]:
>>> from sklearn import metrics
>>> print(metrics.classification_report(twenty_test.target, predicted))

C:\Users\ASUS\.conda\envs\workspace\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          01       0.00      0.00      0.00         0
          02       0.00      0.00      0.00         0
          03       0.00      0.00      0.00         1
          04       0.57      0.28      0.37        90
          05       0.00      0.00      0.00        53
          06       0.02      0.47      0.04        19
          07       0.24      0.29      0.26      1196
          09       0.00      0.00      0.00        13
          10       0.01      0.05      0.02       131
          11       0.25      0.81      0.39        26
          12       0.00      0.00      0.00         0
          13       0.73      1.00      0.84       207
          14       0.00      0.00      0.00         0
          15       0.21      0.74      0.33      1568
          16       0.94      0.60      0.73     19956
          17       0.00      0.75      0.00         4
          18       0.20      0.28      0.23        78
          20       0.00    

In [24]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     1,     0,     0,     0],
       [    1,     1,     0,    25,     2,    28,     0,     2,     0,
            0,     0,     0,     5,     0,    20,     6,     0,     0],
       [    0,     0,     5,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    48,     0,     0,     0],
       [    0,     0,     0,     0,     0,     9,     0,     0,     0,
            0,     0,     0,     0,     0,     8,     2,     0,     0],
       [    0,     0,    11,     0,     0,    97,   350,     0,     0,
            3,     0,     0,     0,   225,   415,    70,    25,     0],

In [25]:
>>> from sklearn.model_selection import GridSearchCV
>>> parameters = {
...     'vect__ngram_range': [(1, 1), (1, 2)],
...     'tfidf__use_idf': (True, False),
...     'clf__alpha': (1e-2, 1e-3),
... }

In [26]:
txt = ["Textile fabrics","Horse"]

docs_test = txt
predicted = text_clf.predict(docs_test)

predicted 


array(['11', '15'], dtype='<U2')

In [27]:
text_clf.fit(twenty_train.data, twenty_train.target)

predicted = text_clf.predict(txt)
predicted

array(['11', '15'], dtype='<U2')

In [28]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
twenty_train.target[gs_clf.predict(['Textile fabrics'])[0]]

KeyError: '11'

In [ ]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
twenty_train.target = gs_clf.predict(['Textile fabrics'])

In [ ]:
gs_clf.best_score_

In [ ]:
for param_name in sorted(parameters.keys()):
...     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))